# RAG 평가 개요
- RAG 평가란 RAG 시스템이 주어진 입력에 대해 얼마나 효과적으로 관련 정보를 검색하고, 이를 기반으로 정확하고 유의미한 응답을 생성하는지를 측정하는 과정이다. 
- **평가 요소**
    - **검색 단계 평가**
        - 입력 질문에 대해 검색된 문서나 정보의 관련성과 정확성을 평가.
    - **생성 단계 평가**
        - 검색된 정보를 기반으로 생성된 응답의 품질, 정확성등을 평가.
- **평가 방법**
    - 온/오프라인 평가
        1. **오프라인 평가**
            - 미리 준비된 데이터셋을 활용하여 RAG 시스템의 성능을 측정한다.
        2. **온라인 평가**
            - 실제 사용자 트래픽과 피드백을 기반으로 시스템의 실시간 성능을 평가한다.
    - 정량적/정성적 평가
        1. 정량적 평가
            - 자동화된 지표를 사용하여 생성된 텍스트의 품질을 평가한다.
        2. 정성적 평가
            - 전문가나 일반 사용자가 직접 생성된 응답의 품질을 평가하여 주관적인 지표를 평가한다.

# [RAGAS](https://www.ragas.io/)
- RAGAS는 RAG 파이프라인을 **정량적 으로 평가하는** 오픈소스 프레임 워크이다. 
- RAGAS 문서: https://docs.ragas.io/en/stable/
## 설치
- `pip install ragas`

In [3]:
import ragas

ragas.__version__

'0.2.15'

## RAGAS 평가 지표 개요
![ragas_score](figures/ragas_score.png)
- **Generation**
    - llm 모델이 생성한 답변에 대한 평가 지표들.
    - **Faithfulness(신뢰성)**
        -  생성된 답변과 검색된 문서(context)간의 관련성을 평가하는 지표
        -  생성된 답변이 주어진 문맥(context)에 얼마나 충실한지를 평가하는 지표로 할루시네이션에 대한 평가로 볼 수있다.
    - **Answer relevancy(답변 적합성)**
        - 생성된 답변과 사용자의 질문간의 관련성을 평가하는 지표
        - 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지를 평가하는 지표.
- **Retrieval**
    -  질문에 대해 검색한 문서(context)들에 대한 평가
    -  **Context Precision(문맥 정밀도)**
        -  검색된 문서(context)들 중 질문과 관련 있는 것들이 **얼마나 상위 순위에 위치하는지** 평가하는 지표.
    -  **Context Recall(문맥 재현률)**
        -  검색된 문서(context)가 정답(ground-truth)의 정보를 얼마나 포함하고 있는지 평가하는 지표.
- 이러한 지표들은 RAG 파이프라인의 성능을 다각도로 평가하는 데 활용된다.
![RAGAS_score2](figures/RAGAS_score2.png)

## 주요 평가지표
### Generation 평가
- LLM이 생성한 답변에 대한 평가
  
#### Faithfulness (신뢰성)
- 생성된 답변이 얼마나 주어진 검색 문서들(context)를 잘 반영해서 생성되었는지 평가한다. 할루시네이션에 대한 평가라고 할 수 있다. 
- 점수범위: **0 ~ 1** (1에 가까울수록 좋음)
- 답변에 포함된 모든 주장이 context에서 얼마나 추출 가능한지를 확인한다.

##### 평가 방법
1. Answer에서 주장 구문(claim statement)들을 생성(추출)한다. (주장이란, 질문(user input)과 관련된 내용)
    - 예) 
        - **질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요? 
        - **LLM 답변**: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 3000만명이다.
2. 각 주장들을 context로 부터 추론 가능한지 판단한다. 이를 바탕으로 faithfulness 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다. .... 한국의 인구는 5000만명이고 서울에 1000만이 살고 있다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 추론가능한 주장.
            - 한국의 인구는 3000만명이다. -> context에서 추론 불가능한 주장.
3. **Faithfulness score** 를 계산한다. 총 주장 수 중에서 context로 부터 추론가능한 주장의 개수.    
    - 예)
        - Faithfulness Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 유추할 수있다.)
    - LLM 답변에서 주장을 추출 하는 것과 각 주장이 context에서 추론 가능한 지를 판단하는 것은 LLM 을 활용한다.
- 공식
    $$
    \text{Faithfulness Score}\;=\;\cfrac{\text{주어진\;context\;에서\;추론할\;수\;있는\;주장의\;개수}}{\text{총\;주장\;개수}}
    $$

#### Answer relevancy (답변 적합성)
- 생성된 답변이 질문(user input)에 얼마나 잘 부합하는 지를 평가한다.
- 점수 범위: -1~1 (1에 가까울수록 좋음)
- LLM이 생성한 답변을 기반으로 질문들을 생성한다. 이렇게 생성한 질문들과 실제 질문(user input)의 embedding vector 간의 **코사인 유사도**를 측정한다.

##### 평가방법
1. LLM이 생성한 답변을 기반으로 질문들을 생성한다.
    - 예) 
        - **LLM** 답변: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
2. 실제 질문과 생성한 질문간의 코사인 유사도를 측정한다. 그 평균이 최종 점수가 된다.
    - 예)
        - **실제 질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요?
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
- 공식
  $$
    \cfrac{1}{N} \sum_{i=1}^{N} \text{cosine\_similarity}(q_{\text{user}_{_i}}, q_{\text{generated}})
  $$

### Retrieval 평가
User input에 대해 Vector store에서 검색한 context에 대한 평가

#### Context Precision
- 사용자가 질문(query)에 대해 검색된 문서들이 답변 생성에 얼마나 유용한지를 평가한다.
- 검색된 K개의 문서(context)들 중 **질문과 관련 있는 문서들이 얼마나 상위 순위**에 있는 지로 평가.
- 점수 범위: 0~1 (1에 가까울수록 좋음)


##### 평가방법

- 공식
$$
 \text{Context\;Precision@K} = \frac{\sum_{k=1}^{K} \left( \text{Precision@k} \times v_k \right)}{\ 상위\;K개\;결과에서의\;관련\;항목\;수}
$$
$$
 \text{Precision@k} = \frac{\text{True\;positive@k}}{(\text{True\;positive@k} + \text{False\;positive@k})} \\
$$
- $\text{True Positive@k}$: 상위 k개의 문서 중 질문과 관련있는 문서의 개수
- $\text{False Positive@k}$: 상위 k개의 문서 중 질문과 관련없는 문서의 개수
- $\text{Precision@k}$: 상위 k개의 문서 중 질문과 관련된 문서들이 차지하는 비율
- K: 검색한 context(문서)의 개수(chuck 수)
- $v_k$: 질문과의 context간의 관련성 여부로 0 또는 1. (0: 관련 없음, 1: 관련 있음)

##### 예시
- 질문과 context 관련성 예
    - 질문: 한국의 수도는 어디이고 인구는 얼마나 되나요?
    - 높은 정밀도 context(관련성 높은 문서의 예)
        - 한국의 수도는 서울이고 인구는 5000명 입니다. 
        - 한국의 수도는 서울입니다.
        - 한국은 동아시아에 위치해 있는 국가로 수도는 서울입니다.
        - 한국의 인구는 5000만명 입니다.
    - 낮은 정밀도 context(관련성 낮은 문서의 예)
        - 한국은 동아시아에 위치한 국가입니다.
        - 한국의 K-pop은 전 세계적으로 유명합니다.
        - 비빔밥, 불고기는 한국의 대표적인 음식입니다.
    - **높은 정밀도의 context이 상위 순위에 위치했으면 높은 점수를 받는다.**
- 점수 계산의 예
  - 상위 5개의 검색 결과 중 1번째, 3번째, 4번째 문서가 관련이 있다고 가정
    ```bash
    Precision@1 = 1/1 = 1.0    # True positive@1/(True positive@1 + False positive@1).  1/1(1번 문서 계산 시에는 1개 문서만 있으므로 분모가 1이 된다.)
    Precision@2 = 1/2 = 0.5     
    Precision@3 = 2/3 ≈ 0.67    
    Precision@4 = 3/4 = 0.75
    Precision@5 = 3/5 = 0.6
    ```
- vk의 값
    - 1번째: $v_1 = 1$
    - 2번째: $v_2 = 0$
    - 3번째: $v_3 = 1$
    - 4번째: $v_4 = 1$
    - 5번째: $v_5 = 0$

- Context Precision@5
$$
\text{Context\;Precision@5} = \frac{(1.0 \times 1) + (0.5 \times 0) + (0.67 \times 1) + (0.75 \times 1) + (0.6 \times 0)}{3} = \frac{1.0 + 0 + 0.67 + 0.75 + 0}{3} ≈ 0.807
$$

#### Context Recall (문맥 재현률)
- 검색된 문서(context)가 얼마나 정답(ground-truth)의 정보를 포함있는 지 평가하는 지표
- 점수 범위: 0~1 (1에 가까울수록 좋음)
- **정답(ground truth)의 각 주장(claim)이 검색된 context와 얼마나 일치**하는지 계산함.

##### 평가방법
1. **정답**에서 주장 문장(claim statement)들을 생성(추출)한다.
    - 예) 
        - **정답**: 한국의 수도는 서울이고 인구수는 5000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 5000만명이다.
2. 각 주장 문장(claim statement)의 정보를 검색된 context들에서 찾을 수 있는지 판별한다. 이를 바탕으로 context recall 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 찾을 수 있다.
            - 한국의 인구는 5000만명이다. -> context에서 찾을 수 없다.
3. **Context Recall Score** 를 계산한다. 총 주장 수 중에서 context로 부터 찾을 수 있는 주장의 개수.
    - 예)
        - Context Recall Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 찾을 수 있다.)

- 공식
    $$
    \text{Context Recall Score}\;=\;\cfrac{\text{GT의\;주장\;중\;주어진\;context\;에서\;찾을\;수\;있는\;주장의\;개수}}{\text{GT의\;총\;주장\;개수}}
    $$ 

# RAGAS 평가 실습

In [2]:
# %pip install ragas

In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(model="gpt-4.1-mini")
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-large"
)


In [4]:
## ChromaDB와 연결 하고 vector store 생성
from langchain_chroma import Chroma

COLLECTION_NAME = 'olympic_info'
PERSIST_DIRECTORY = 'vector_store/chroma/olympic_info'

vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)
print(vector_store._collection.count())

61


In [5]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("data/olympic.txt", encoding="utf-8")
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

docs = loader.load_and_split(splitter)
# vector_store.add_documents(docs)

In [6]:
print(vector_store._collection.count())

61


## RAG Chain 구성
- Vector Store 연결
- Chain 응답 결과
    - 평가를 위해 **Vector Store에서 검색한 context**들과 **LLM 응답**이 출력되도록 chain을 구성한다.

In [7]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document

retriever = vector_store.as_retriever()

template = """# Instruction:
당신은 정확한 정보 제공을 우선시하는 인공지능 어시스턴트입니다.
주어진 Context에 포함된 정보만 사용해서 질문에 답변하세요.
Context에 질문에 대한 명확한 정보가 있는 경우 그 내용을 바탕으로 답변하세요.
Context에 질문에 대한 명확한 정보없을 경우 "정보가 부족해서 답을 알 수 없습니다." 라고 대답합니다.
절대 Context에 없는 내용을 추측하거나 일반 상식을 이용해 답을 만들어서 대답하지 않습니다.

# Context:
{context}

# 질문:
{query}
"""
prompt_template = PromptTemplate(template=template)

In [8]:
def format_docs(src_dict:dict)->str:
    """
    Vector_store에 조회한 문서들에서 내용(page_content)만 추출해서 str으로 합쳐서 반환.
    
    Args:
        src_dict(dict): {"context":list[Document],  "query":"사용자질문"}
    Returns:
        str: 각 문서의 내용을 "\n\n"으로 연결한 string
    """
    docs = src_dict["context"]
    return "\n\n".join(doc.page_content for doc in docs)

def format_docs_list(src_dict:dict)->list[str]:
    """
    Vector_store에 조회한 문서들에서 내용(page_content)만 추출해서 list로 묶어서 반환.
    
    Args:
        src_dict(dict): {"context":list[Document],  "query":"사용자질문"}
    Returns:
        list[str]: 각 문서의 내용을 담은 list
    """
    return [doc.page_content for doc in src_dict["context"]]

In [9]:
# docs = retriever.invoke("올림픽 논란")
# docs
# format_docs({"context":docs})
# format_docs_list({"context":docs})

In [10]:
# 평가를 위한 Rag Chain을 구성
## chain의 응답: LLM 응답, Retriever가 검색한 context들 
rag_chain = (
    RunnablePassthrough()  # dict | dict 를 LCEL로 연결하기 위해 하는일 없는 Runnable(RunnablePassthrough) 을 추가
    | {"context":retriever, "query":RunnablePassthrough()} # retrieve
    | {
        "source_context":format_docs_list, # list[Document] -> list[str]
        "llm_answer": {
                        "context":format_docs, # list[Document] -> str(문서\n\n문서\n\n...)
                        "query": lambda x : x["query"]    # query만 추출
                    } | prompt_template | model | StrOutputParser()
    } # 응답처리 - 입력: {"context":검색문서들, "query":"사용자질문"} 
      #          - 출력: {"source_context":검색한 문서들(list[str]), "llm_answer":LLM응답(str)}
      #             -> 응답처리 출력이 chain의 최종 출력
    
)

In [11]:
user_input = "국제 올림픽 위원회에 대해서 설명해주세요."
response = rag_chain.invoke(user_input)

In [12]:
print(type(response), response.keys())

<class 'dict'> dict_keys(['source_context', 'llm_answer'])


In [13]:
llm_answer = response['llm_answer']
print(llm_answer)

국제 올림픽 위원회(IOC)는 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있습니다. 올림픽 활동은 많은 국가, 국제 경기 연맹과 협회, 미디어 파트너, 선수, 직원, 심판 등 모든 관련자가 올림픽 헌장을 지키는 것을 말하며, 크게 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 올림픽 조직 위원회(OCOG)로 구성됩니다.

IOC가 의사 결정 기구로서 올림픽 개최 도시를 선정하고, 각 올림픽 대회에서 열리는 종목도 결정합니다. 또한 IOC는 올림픽과 관련된 여러 권한을 가지고 있습니다. IOC 위원장 중에는 에이버리 브런디지와 후안 안토니오 사마란치가 있으며, 이들은 각각 정치적 문제 및 부패와 관련해 비판을 받은 바 있습니다.


In [14]:
context = response["source_context"]
context

['국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.\n- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)',
 '올림픽은 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 각 올림픽의 위원회(예-벤쿠버동계올림픽조직위원회)로 구성된다. 의사 결정 기구인 IOC는 올림픽 개최 도시를 선정하며, 각 올림픽 대회마다 열리는 올림픽 종목도 IOC에서 결정한다. 올림픽 경기 개최 도시는 경기 축하 의식이 올림픽 헌장에 부합하도록 조직하고 기금을 마련해야 한다. 올림픽 축하 행사로는 여러 의식과 상징을 들 수 있는데 올림픽기나 성화가 그 예이다.',
 '- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.\n- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.\n올림픽의 공식언어는 프랑스어와 영어와 개최국의 공용어이다. 모든 선언(예를 들어서 개막식 때 각국 소개를 할 때)들은 세 언어가 모두 나오거나 영어나 프랑스어 중에서 한 언어로만 말하기도 한다. 개

## 평가

In [15]:
# 평가데이터셋: user_input, response(정답)
user_input = "국제 올림픽 위원회에 대해 설명해주세요."
reference = "국제올림픽위원회(IOC)는 올림픽 활동을 통솔하는 기구로, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 역할을 수행한다."

resp = rag_chain.invoke(user_input)
retrieved_context = resp['source_context']  # RAG에서 검색된 문서
response = resp['llm_answer']  # RAG의 답변

In [16]:
from ragas import SingleTurnSample, EvaluationDataset

# RAGAS의 평가 데이터셋의 구성
# - user_input: 사용자 입력(질문)
# - retrieved_context: RAG에서 검색한 context(문서)들
# - response: llm의 출력(응답)
# - reference: 정답.

eval_sample1 = SingleTurnSample(  # 1개의 평가 데이터을 생성할 때 사용.
    user_input=user_input,
    retrieved_contexts=retrieved_context,
    response=response,
    reference=reference
)
# 평가용 Dataset을 생성
eval_dataset = EvaluationDataset(samples=[eval_sample1])

In [17]:
eval_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [21]:
# eval_dataset을 DataFrame으로 변환
eval_dataset.to_pandas()

,user_input,retrieved_contexts,response,reference
0,국제 올림픽 위원회에 대해 설명해주세요.,"[국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 •...","국제 올림픽 위원회(IOC)는 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 ...","국제올림픽위원회(IOC)는 올림픽 활동을 통솔하는 기구로, 올림픽 개최 도시 선정,..."


In [22]:
##########################
# 평가
from ragas.metrics import (
    LLMContextRecall, # Context recall을 계산하는 **평가함수**
    LLMContextPrecisionWithReference, # Context Precision
    Faithfulness,     # Faithfulness 
    AnswerRelevancy,  # AnswerRelevancy
)
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
# Langchain으로 생성된 LLM 모델, 임베딩모델 객체를 RAGAS에서 사용할 있도록 wrapping하는 클래스.
# 모든 평가 지표들이 LLM 모델을 사용한다. 

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# 평가 함수(객체)에 넣어줄 LLM 모델, Embedding 모델 생성.
llm_model = ChatOpenAI(model="gpt-4.1-mini")
e_model = OpenAIEmbeddings(model="text-embedding-3-large")

eval_model = LangchainLLMWrapper(llm_model)
eval_embedding_model = LangchainEmbeddingsWrapper(e_model)

## 평가지표 객체를 생성 -> 리스트에 묶어준다.
metrics = [
    LLMContextRecall(llm=eval_model),
    LLMContextPrecisionWithReference(llm=eval_model),
    Faithfulness(llm=eval_model),
    AnswerRelevancy(llm=eval_model)
]

# 평가지표 객체를 이용해서 평가
eval_result = evaluate(dataset=eval_dataset, metrics=metrics)

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
# context recall: 검색 context와 정답간의 연관성
# context precision: 검색 context와 사용자 질문간의 연관성
# faithfulness: LLM 답변과 검색 context간의 연관성
# answer relevancy: LLM 답변과 사용자 질문간의 연관성.
print(type(eval_result))
eval_result

<class 'ragas.dataset_schema.EvaluationResult'>


{'context_recall': 1.0000, 'llm_context_precision_with_reference': 0.8333}

In [25]:
eval_result.to_pandas()

,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,faithfulness,answer_relevancy
0,국제 올림픽 위원회에 대해 설명해주세요.,"[국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 •...","국제 올림픽 위원회(IOC)는 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 ...","국제올림픽위원회(IOC)는 올림픽 활동을 통솔하는 기구로, 올림픽 개최 도시 선정,...",1.0,0.833333,0.666667,0.830367


# 평가 데이터 셋 만들기

1. 문서를 Loading하고 split하여 Context 들을 만든다.
2. Context들 중 평가에 사용할 것을 Random하게 선택한다.
3. 선택된 context를 기반으로 LLM 모델을 이용해서 질문과 답변을 생성한다. 
4. 생성된 질문과 답변을 검토하여 품질을 높인다. 
   - 질문과 답변 자체가 맞는지 검사한다.
   - 나올만한 질문인 지 확인한다.

- **생성된 질문-답변의 질이 낮으면 좋은 평가를 할 수 없다.**
    - 질문-답변 생성시 사용하는 LLM 모델은 성능이 좋은 것을 사용해야 한다. 
    - 생성된 질문-답변을 사람이 검토해서 품질을 더 높여야 한다.

## 데이터셋에 포함될 내용
- Dataset을 생성할 때는 다음 항목이 들어가야 한다.
    -  **user_input**: 질문. `string`
    -  **reference**: 정답. `string`
    -  qa_context : context. `str` - QA를 만들 때 어떤 context를 사용했는지 값. 생성된 데이터셋을 확인하기 위한 것으로 **평가시에는 사용하지 않는다.**
-  평가할 때 추가할 것
    - **response**: RAG 시스템에서 질문에 대해 LLM 모델이 생성한 답변. `string`
    - **retrieved_contexts**: RAG 시스템에서 질문에 대해 Vector DB에서 검색한 context들. `list(string)`

In [29]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser

# from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.documents import Document

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from pydantic import BaseModel, Field

import random
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# Context 를 바탕으로 질문과 답변을 생성
# 생성된 질문을 rag_chain에 넣어서 context와 llm 응답을 생성.

In [ ]:
# context 생성
DOC_PATH = "data/olympic.txt"

loader = TextLoader(DOC_PATH, encoding='utf-8')
# Vector DB에 저장한 context와 동일하게 chunking(split)을 한다.
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

docs = loader.load_and_split(splitter)
len(docs) #list[Document]

61

In [36]:
# docs 중에서 평가데이터를 생성할 때 사용할 문서 K개를 추출
#### 실제 평가 데이터셋을 생성할 때는 모든 문서를 다 사용. 
total_samples = 5 # 5개 context만 사용

index_list = list(range(len(docs)))
random.shuffle(index_list)

eval_context = [] # 추출한 context들을 담을 리스트
while len(eval_context) < total_samples:
    index = index_list.pop()
    context = docs[index].page_content
    if len(context) > 200: # 200글자 이상인 것만 사용.
        eval_context.append(context)

eval_context

["2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Programme Commission)를 설립했다. 여기서는 올림픽 종목과 올림픽 종목이 아닌 스포츠를 모두 재검토하는 일을 한다. 이 위원회의 목표는 올림픽 종목에 더 체계적으로 다가가는 것이다. 위원회에서는 우선적으로 올림픽 종목으로 포함되기 위해서는 7개의 기준을 충족시켜야 한다고 말한다. 이 7개의 기준은 역사, 전통, 보편성, 인기도와 잠재성, 선수의 건강, 연맹의 스포츠를 관리할만한 능력, 스포츠를 여는 데에 필요한 비용이다. 예를 들면 2012년 하계 올림픽의 정식종목 후보에 7개 조건을 포함한 비(非)올림픽 스포츠가 올랐고 그 내용은, 골프, 가라테, 럭비, 인라인 스케이팅, 스쿼시였다. 이 스포츠들은 IOC 상임이사회에서 재검토되어 2005년 7월에 열린 싱가포르 총회에서 최종 결정하기로 했다. 결국 5개 중 2개(가라테와 스쿼시) 가 최종 후보로 올라왔으나 가라테와 스쿼시 둘 다",
 '현재 이란 정부는 이스라엘과의 어떤 경기 경쟁이든 피하고 있다. 2008년 하계 올림픽 때 이란의 수영 선수는 이스라엘 수영 선수와 같이 경기한다는 이유로 경기를 포기했으며, 2004년 하계 올림픽에서도 이란의 유도 선수는 이스라엘 선수와 경기한다는 일정이 잡혔을 때 경기를 포기했다. 이 선수는 공식적으로는 시합전에 계체량을 재서 체중이 초과되어 실격 되었으나 이란정부로부터 125,000달러나 되는 돈을 받았다고 한다.',
 '2016년까지 올림픽은 23개국 44개 도시에서 열렸으며 유럽과 북아메리카대륙 이외의 대륙에서는 고작 8번 밖에 개최하지 못했다. 1988년 하계 올림픽이 대한민국의 서울에서 열린것을 시작으로 그 후 아시아와 오세아니아 대륙에서 올림픽이 4번이나 열렸으며, 이는 그 이전의 현대 올림픽사와 비교해보면 엄청나게 늘어난 수치였다. 2016년 하계 올림픽이 개최된 브라질의 리우데자네이루는 남미에서 열리는 첫 번째 올림픽이다. 아직 아프리카에서는 올림픽이 한 번도 

In [40]:
# Context -> 질문과 답변을 생성하는 Chain
# JsonOutputParser를 사용.
## 스키마 생성.
### LLM으로 부터 응답받을 값들을 설계.
class EvalSchema(BaseModel):
    user_input: str = Field(..., description="사용자 질문") # 스키마 변수에 대한 설정. `...` 시작: 필수
    reference: str = Field(..., description="user_input(사용자 질문)에 대한 정답")
    qa_context: str = Field(..., description="질문, 답변 쌍을 만들때 참조한 context. 입력된 context를 수정하지 않고 그대로 넣는다.")

parser = JsonOutputParser(pydantic_object=EvalSchema)

template = """# Instruction:
당신은 RAG 평가를 위해 질문과 정답 쌍을 생성하는 인공지능 비서입니다.
다음 [Context] 에 문서가 주어지면 해당 문서를 기반으로 {num_questions}개 질문-정답 쌍을 생성하세요. 

질문과 정답을 생성한 후 Output Indicator의 format으로 출력합니다.
질문은 반드시 Context 문서에 있는 정보를 바탕으로 생성해야 합니다. Context에 없는 내용을 가지고 질문-정답을 절대 만들면 안됩니다.
올림픽에 관심있는 일반 사용자 관점의 자연스러운 질문을 작성합니다.
질문은 간결하게 작성합니다.
하나의 질문에는 한 가지씩만 내용만 작성합니다.
질문을 만들 때 "제공된 문맥에서", "문서에 설명된 대로", "주어진 문서에 따라" 또는 이와 유사한 말을 하지 마세요.
정답은 반드시 Context에 있는 정보를 바탕으로 작성합니다. 없는 내용을 추가하지 않습니다.
질문과 정답을 만들고 그 내용이 Context에 있는 항목인지 다시 한번 확인합니다.
생성된 질문-답변 쌍은 반드시 dictionary 형태로 정의하고 list로 묶어서 반환해야 합니다.
질문-답변 쌍은 반드시 {num_questions}개를 만들어야 합니다.

#Context:
{context}

Output Indicator:
{format_instructions}
"""

prompt_template = PromptTemplate(
    template=template, 
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
eval_model = ChatOpenAI(model="gpt-4.1")
eval_dataset_chain = prompt_template | eval_model | parser

In [43]:
eval_dataset_chain.invoke({"context":eval_context[0], "num_questions":3})

[{'user_input': 'IOC가 올림픽 종목 선정 시 고려하는 기준은 무엇인가요?',
  'reference': '역사, 전통, 보편성, 인기도와 잠재성, 선수의 건강, 연맹의 스포츠를 관리할만한 능력, 스포츠를 여는 데에 필요한 비용의 7가지 기준이 있습니다.',
  'qa_context': "2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Programme Commission)를 설립했다. 여기서는 올림픽 종목과 올림픽 종목이 아닌 스포츠를 모두 재검토하는 일을 한다. 이 위원회의 목표는 올림픽 종목에 더 체계적으로 다가가는 것이다. 위원회에서는 우선적으로 올림픽 종목으로 포함되기 위해서는 7개의 기준을 충족시켜야 한다고 말한다. 이 7개의 기준은 역사, 전통, 보편성, 인기도와 잠재성, 선수의 건강, 연맹의 스포츠를 관리할만한 능력, 스포츠를 여는 데에 필요한 비용이다. 예를 들면 2012년 하계 올림픽의 정식종목 후보에 7개 조건을 포함한 비(非)올림픽 스포츠가 올랐고 그 내용은, 골프, 가라테, 럭비, 인라인 스케이팅, 스쿼시였다. 이 스포츠들은 IOC 상임이사회에서 재검토되어 2005년 7월에 열린 싱가포르 총회에서 최종 결정하기로 했다. 결국 5개 중 2개(가라테와 스쿼시) 가 최종 후보로 올라왔으나 가라테와 스쿼시 둘 다"},
 {'user_input': '2012년 하계 올림픽 정식 종목 후보에 올랐던 비올림픽 스포츠는 어떤 종목이었나요?',
  'reference': '골프, 가라테, 럭비, 인라인 스케이팅, 스쿼시가 있었습니다.',
  'qa_context': "2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Programme Commission)를 설립했다. 여기서는 올림픽 종목과 올림픽 종목이 아닌 스포츠를 모두 재검토하는 일을 한다. 이 위원회의 목표는 올림픽 종목에 더 체계적으로 다가가는 것이다. 위원회에서는 우선적으로 올림픽 종목으로 포함되기 위해서는 7개의 기준을

In [ ]:
# context당 5개씩 질문을 생성
eval_dataset_list = []
num_questions = 5
for context in eval_context:
    eval_data = eval_dataset_chain.invoke({"context":context, "num_questions":num_questions})
    eval_dataset_list.extend(eval_data)

In [69]:
len(eval_dataset_list)

25

In [68]:
################# 생성된 질문 쌍을 반드시 검토 해야 한다.#################
eval_dataset_list[3]

{'user_input': '올림픽 종목 포함 기준 중 하나로 언급된 선수 관련 사항은 무엇인가요?',
 'reference': '선수의 건강이 기준 중 하나입니다.',
 'qa_context': "2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Programme Commission)를 설립했다. 여기서는 올림픽 종목과 올림픽 종목이 아닌 스포츠를 모두 재검토하는 일을 한다. 이 위원회의 목표는 올림픽 종목에 더 체계적으로 다가가는 것이다. 위원회에서는 우선적으로 올림픽 종목으로 포함되기 위해서는 7개의 기준을 충족시켜야 한다고 말한다. 이 7개의 기준은 역사, 전통, 보편성, 인기도와 잠재성, 선수의 건강, 연맹의 스포츠를 관리할만한 능력, 스포츠를 여는 데에 필요한 비용이다. 예를 들면 2012년 하계 올림픽의 정식종목 후보에 7개 조건을 포함한 비(非)올림픽 스포츠가 올랐고 그 내용은, 골프, 가라테, 럭비, 인라인 스케이팅, 스쿼시였다. 이 스포츠들은 IOC 상임이사회에서 재검토되어 2005년 7월에 열린 싱가포르 총회에서 최종 결정하기로 했다. 결국 5개 중 2개(가라테와 스쿼시) 가 최종 후보로 올라왔으나 가라테와 스쿼시 둘 다"}

In [70]:
# DataFrame으로 생성
import pandas as pd
eval_df = pd.DataFrame(eval_dataset_list)
eval_df.head()

,user_input,reference,qa_context
0,올림픽 프로그램 위원회는 언제 설립되었나요?,2004년 10월과 11월에 설립되었습니다.,2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Pr...
1,올림픽 종목으로 선정되기 위해 필요한 기준은 몇 가지인가요?,7가지 기준이 필요합니다.,2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Pr...
2,2012년 하계 올림픽 정식종목 후보로 올랐던 비올림픽 스포츠에는 어떤 것들이 있었나요?,"골프, 가라테, 럭비, 인라인 스케이팅, 스쿼시 등이 있었습니다.",2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Pr...
3,올림픽 종목 포함 기준 중 하나로 언급된 선수 관련 사항은 무엇인가요?,선수의 건강이 기준 중 하나입니다.,2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Pr...
4,2012년 하계 올림픽 종목 최종 후보로 남았던 스포츠는 무엇인가요?,가라테와 스쿼시가 최종 후보로 남았습니다.,2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Pr...


In [72]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_input  25 non-null     object
 1   reference   25 non-null     object
 2   qa_context  25 non-null     object
dtypes: object(3)
memory usage: 732.0+ bytes


In [75]:
#### 평가할 RAG chain(rag_chain)을 이용해서 context 문서들과 llm 응답을 받아서 
##        평가 데이터셋을 완성.
# user_input -> (rag_chain) -> {llm_answer, source_context}
context_list = []  # source_context들을 저장할 리스트
response_list = [] # llm_answer드을 저장할 리스트

for user_input in eval_df['user_input']:
    resp = rag_chain.invoke(user_input)
    context_list.append(resp['source_context'])
    response_list.append(resp['llm_answer'])

In [76]:
len(context_list), len(response_list)

(25, 25)

In [77]:
eval_df.loc[0, 'user_input']

'올림픽 프로그램 위원회는 언제 설립되었나요?'

In [78]:
response_list[0]

'올림픽 프로그램 위원회는 2004년 10월과 11월에 설립되었습니다.'

In [79]:
context_list[0]

["2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Programme Commission)를 설립했다. 여기서는 올림픽 종목과 올림픽 종목이 아닌 스포츠를 모두 재검토하는 일을 한다. 이 위원회의 목표는 올림픽 종목에 더 체계적으로 다가가는 것이다. 위원회에서는 우선적으로 올림픽 종목으로 포함되기 위해서는 7개의 기준을 충족시켜야 한다고 말한다. 이 7개의 기준은 역사, 전통, 보편성, 인기도와 잠재성, 선수의 건강, 연맹의 스포츠를 관리할만한 능력, 스포츠를 여는 데에 필요한 비용이다. 예를 들면 2012년 하계 올림픽의 정식종목 후보에 7개 조건을 포함한 비(非)올림픽 스포츠가 올랐고 그 내용은, 골프, 가라테, 럭비, 인라인 스케이팅, 스쿼시였다. 이 스포츠들은 IOC 상임이사회에서 재검토되어 2005년 7월에 열린 싱가포르 총회에서 최종 결정하기로 했다. 결국 5개 중 2개(가라테와 스쿼시) 가 최종 후보로 올라왔으나 가라테와 스쿼시 둘 다",
 '올림픽은 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 각 올림픽의 위원회(예-벤쿠버동계올림픽조직위원회)로 구성된다. 의사 결정 기구인 IOC는 올림픽 개최 도시를 선정하며, 각 올림픽 대회마다 열리는 올림픽 종목도 IOC에서 결정한다. 올림픽 경기 개최 도시는 경기 축하 의식이 올림픽 헌장에 부합하도록 조직하고 기금을 마련해야 한다. 올림픽 축하 행사로는 여러 의식과 상징을 들 수 있는데 올림픽기나 성화가 그 예이다.',
 '올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세

In [81]:
eval_df['response'] = response_list
eval_df['retrieved_contexts'] = context_list

In [82]:
eval_df.head()

,user_input,reference,qa_context,response,retrieved_contexts
0,올림픽 프로그램 위원회는 언제 설립되었나요?,2004년 10월과 11월에 설립되었습니다.,2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Pr...,올림픽 프로그램 위원회는 2004년 10월과 11월에 설립되었습니다.,[2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic P...
1,올림픽 종목으로 선정되기 위해 필요한 기준은 몇 가지인가요?,7가지 기준이 필요합니다.,2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Pr...,올림픽 종목으로 선정되기 위해 필요한 기준은 7가지입니다.,[2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic P...
2,2012년 하계 올림픽 정식종목 후보로 올랐던 비올림픽 스포츠에는 어떤 것들이 있었나요?,"골프, 가라테, 럭비, 인라인 스케이팅, 스쿼시 등이 있었습니다.",2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Pr...,"2012년 하계 올림픽 정식종목 후보로 올랐던 비올림픽 스포츠는 골프, 가라테, 럭...",[결국 5개 중 2개(가라테와 스쿼시) 가 최종 후보로 올라왔으나 가라테와 스쿼시 ...
3,올림픽 종목 포함 기준 중 하나로 언급된 선수 관련 사항은 무엇인가요?,선수의 건강이 기준 중 하나입니다.,2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Pr...,"올림픽 종목 포함 기준 중 하나로 언급된 선수 관련 사항은 ""프로 선수의 참가 불허...",[올림픽 경기 종목\n올림픽 경기 종목은 총 33개부문 52개 종목에서 약 400개...
4,2012년 하계 올림픽 종목 최종 후보로 남았던 스포츠는 무엇인가요?,가라테와 스쿼시가 최종 후보로 남았습니다.,2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic Pr...,2012년 하계 올림픽 종목 최종 후보로 남았던 스포츠는 가라테와 스쿼시입니다.,[결국 5개 중 2개(가라테와 스쿼시) 가 최종 후보로 올라왔으나 가라테와 스쿼시 ...


In [ ]:
###################################
#  RAGAS 의 EvaluationDataset 생성
###################################
# feature로 "user_input", "response", "retrieved_contexts", "referece"를 가진 DataFrame으로부터
#     EvaluationDataset생성
evaludation_dataset = EvaluationDataset.from_pandas(eval_df)
evaludation_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=25)

In [84]:
#################################
# 평가
#################################
eval_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
eval_embedding = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-large"))
metrics = [
    LLMContextPrecisionWithReference(llm=eval_llm),
    LLMContextRecall(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]

eval_result = evaluate(dataset=evaludation_dataset, metrics=metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [85]:
eval_result

{'llm_context_precision_with_reference': 0.8300, 'context_recall': 0.9600, 'faithfulness': 1.0000, 'answer_relevancy': 0.6754}

In [86]:
eval_result.to_pandas()

,user_input,retrieved_contexts,response,reference,llm_context_precision_with_reference,context_recall,faithfulness,answer_relevancy
0,올림픽 프로그램 위원회는 언제 설립되었나요?,[2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic P...,올림픽 프로그램 위원회는 2004년 10월과 11월에 설립되었습니다.,2004년 10월과 11월에 설립되었습니다.,1.000000,1.0,1.0,0.798414
1,올림픽 종목으로 선정되기 위해 필요한 기준은 몇 가지인가요?,[2004년 10월과 11월에 IOC는 '올림픽 프로그램 위원회'(Olympic P...,올림픽 종목으로 선정되기 위해 필요한 기준은 7가지입니다.,7가지 기준이 필요합니다.,1.000000,1.0,1.0,0.805069
2,2012년 하계 올림픽 정식종목 후보로 올랐던 비올림픽 스포츠에는 어떤 것들이 있었나요?,[결국 5개 중 2개(가라테와 스쿼시) 가 최종 후보로 올라왔으나 가라테와 스쿼시 ...,"2012년 하계 올림픽 정식종목 후보로 올랐던 비올림픽 스포츠는 골프, 가라테, 럭...","골프, 가라테, 럭비, 인라인 스케이팅, 스쿼시 등이 있었습니다.",1.000000,1.0,1.0,0.666502
3,올림픽 종목 포함 기준 중 하나로 언급된 선수 관련 사항은 무엇인가요?,[올림픽 경기 종목\n올림픽 경기 종목은 총 33개부문 52개 종목에서 약 400개...,"올림픽 종목 포함 기준 중 하나로 언급된 선수 관련 사항은 ""프로 선수의 참가 불허...",선수의 건강이 기준 중 하나입니다.,0.000000,0.0,1.0,0.577669
4,2012년 하계 올림픽 종목 최종 후보로 남았던 스포츠는 무엇인가요?,[결국 5개 중 2개(가라테와 스쿼시) 가 최종 후보로 올라왔으나 가라테와 스쿼시 ...,2012년 하계 올림픽 종목 최종 후보로 남았던 스포츠는 가라테와 스쿼시입니다.,가라테와 스쿼시가 최종 후보로 남았습니다.,1.000000,1.0,1.0,0.757519
5,이란 정부는 이스라엘과의 경기에서 어떤 입장을 취하고 있나요?,[현재 이란 정부는 이스라엘과의 어떤 경기 경쟁이든 피하고 있다. 2008년 하계 ...,이란 정부는 이스라엘과의 어떤 경기 경쟁이든 피하고 있습니다.,이란 정부는 이스라엘과의 어떤 경기 경쟁이든 피하고 있습니다.,1.000000,1.0,1.0,0.719376
6,2008년 하계 올림픽에서 이란의 수영 선수는 왜 경기를 포기했나요?,[현재 이란 정부는 이스라엘과의 어떤 경기 경쟁이든 피하고 있다. 2008년 하계 ...,2008년 하계 올림픽에서 이란의 수영 선수는 이스라엘 수영 선수와 같이 경기한다는...,이란의 수영 선수는 이스라엘 수영 선수와 같이 경기한다는 이유로 경기를 포기했습니다.,1.000000,1.0,1.0,0.788715
7,2004년 하계 올림픽에서 이란의 유도 선수는 어떤 이유로 경기를 포기했나요?,[현재 이란 정부는 이스라엘과의 어떤 경기 경쟁이든 피하고 있다. 2008년 하계 ...,2004년 하계 올림픽에서 이란의 유도 선수는 이스라엘 선수와 경기한다는 일정이 잡...,이스라엘 선수와 경기하는 일정이 잡혔을 때 경기를 포기했습니다.,1.000000,1.0,1.0,0.681521
8,2004년 하계 올림픽에서 이란의 유도 선수에게 공식적으로 적용된 실격 사유는 무엇...,[현재 이란 정부는 이스라엘과의 어떤 경기 경쟁이든 피하고 있다. 2008년 하계 ...,2004년 하계 올림픽에서 이란의 유도 선수에게 공식적으로 적용된 실격 사유는 시합...,공식적으로는 시합 전에 계체량을 재서 체중이 초과되어 실격되었습니다.,1.000000,1.0,1.0,0.673546
9,2004년 하계 올림픽 당시 이란의 유도 선수가 이란정부로부터 받은 금전적 보상은 ...,[현재 이란 정부는 이스라엘과의 어떤 경기 경쟁이든 피하고 있다. 2008년 하계 ...,"2004년 하계 올림픽 당시 이란의 유도 선수는 이란정부로부터 125,000달러를 ...","이란정부로부터 125,000달러를 받았습니다.",1.000000,1.0,1.0,0.806618


# 생성한 평가 데이터셋을 Huggingface에 저장/로드

In [ ]:
# %pip install transformers

In [89]:
from huggingface_hub import login
import os
HF_KEY = os.getenv("HUGGINGFACE_API_KEY")
login(HF_KEY)

In [91]:
# %pip install datasets
from datasets import Dataset
eval_dataset = Dataset.from_pandas(eval_df)
eval_dataset

Dataset({
    features: ['user_input', 'reference', 'qa_context', 'response', 'retrieved_contexts'],
    num_rows: 25
})

In [93]:
# huggingface에  upload
eval_dataset.push_to_hub("wiki_olympic_rag_eval_dataset") # dataset_id : 계정/id

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kgmyh/wiki_olympic_rag_eval_dataset/commit/87324ab3471274908e541e12644214b2792d7dcf', commit_message='Upload dataset', commit_description='', oid='87324ab3471274908e541e12644214b2792d7dcf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/kgmyh/wiki_olympic_rag_eval_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='kgmyh/wiki_olympic_rag_eval_dataset'), pr_revision=None, pr_num=None)

In [94]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
load_eval_dataset = load_dataset("kgmyh/wiki_olympic_rag_eval_dataset")
load_eval_dataset

README.md:   0%|          | 0.00/535 [00:00<?, ?B/s]

c:\Users\Playdata\miniconda3\envs\lang_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\datasets--kgmyh--wiki_olympic_rag_eval_dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/33.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['user_input', 'reference', 'qa_context', 'response', 'retrieved_contexts'],
        num_rows: 25
    })
})